In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
from google.colab import files


uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
import os,glob
import numpy as np
import cv2

import glob
import pickle
import tensorflow as tf
import argparse
import re
import datetime
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,ConvLSTM2D,LayerNormalization,GlobalAveragePooling2D,DepthwiseConv2D,SeparableConv2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler 
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure
from tensorflow.keras.layers import Layer

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tawsifurrahman/tuberculosis-tb-chest-xray-dataset

In [ ]:
!unzip /content/tuberculosis-tb-chest-xray-dataset.zip

In [ ]:
normal_files = []
normal_dir = "/content/TB_Chest_Radiography_Database/Normal"
dir1 = os.path.join(normal_dir,"*.png")
normal_files = glob.glob(dir1)

In [ ]:
tb_files = []
tb_dir = "/content/TB_Chest_Radiography_Database/Tuberculosis"
dir1 = os.path.join(tb_dir,"*.png")
tb_files = glob.glob(dir1)

In [ ]:
print(len(normal_files),len(tb_files))

In [ ]:
normal_files.sort()
tb_files.sort()

In [ ]:
normal_files[:10]

In [ ]:

tb_files[:10]

In [ ]:
2800 * 0.2

In [ ]:

3500-1260

In [ ]:
1260-700

#FOLD 5

In [ ]:

# normal_files_test = normal_files[:700]
# normal_files_train = normal_files[1260:3500]
# normal_val = normal_files[700:1260]
# tb_files_test = tb_files[:700]
# tb_files_train = tb_files[1260:3500]
# tb_val = tb_files[700:1260]


#FOLD 4

In [ ]:
normal_files_test = normal_files[700:1400]
normal_files_train = normal_files[:700]
normal_val = normal_files[1400:1960]
normal_files_train.extend(normal_files[1960:3500])
tb_files_test = tb_files[700:1400]
tb_files_train = tb_files[:700]
tb_val = tb_files[1400:1960]
tb_files_train.extend(tb_files[1960:3500])

In [ ]:
1400-560

#FOLD 3

In [ ]:
# normal_files_test = normal_files[1400:2100]
# normal_files_train = normal_files[:840]
# normal_val = normal_files[840:1400]
# normal_files_train.extend(normal_files[2100:3500])
# tb_files_test = tb_files[1400:2100]
# tb_files_train = tb_files[:840]
# tb_val = tb_files[840:1400]
# tb_files_train.extend(tb_files[2100:3500])

#FOLD 2

In [ ]:
# normal_files_test = normal_files[2100:2800]
# normal_files_train = normal_files[:1540]
# normal_val = normal_files[1540:2100]
# normal_files_train.extend(normal_files[2800:3500])
# tb_files_test = tb_files[2100:2800]
# tb_files_train = tb_files[:2100]
# tb_val = tb_files[1540:2100]
# tb_files_train.extend(tb_files[2800:3500])

In [ ]:
# 2800-560

#FOLD 1


In [ ]:
# normal_files_test = normal_files[2800:3500]
# normal_files_train = normal_files[560:2800]
# normal_val = normal_files[0:560]
# tb_files_test = tb_files[2800:3500]
# tb_files_train = tb_files[560:2800]
# tb_val = tb_files[0:560]


In [ ]:
tb_files_test[:10]

In [ ]:
len(normal_files_train)

In [ ]:

train_dic = {}
test_dic = {}
val_dic = {}
for i in range(len(normal_files_train)):
    train_dic[normal_files_train[i]] = [0,1]
for i in range(len(tb_files_train)):
    train_dic[tb_files_train[i]] = [1,0]

for i in range(len(normal_val)):
    val_dic[normal_val[i]] = [0,1]
for i in range(len(tb_val)):
    val_dic[tb_val[i]] = [1,0]       
    
for i in range(len(normal_files_test)):
    test_dic[normal_files_test[i]] = [0,1]
for i in range(len(tb_files_test)):
    test_dic[tb_files_test[i]] = [1,0]

In [ ]:

train_dic

In [ ]:
import random
l_train = list(train_dic.items())
random.Random(4).shuffle(l_train)
l_train

In [ ]:
import random
l_val = list(val_dic.items())
random.Random(4).shuffle(l_val)
l_val

In [ ]:
import random
l_test = list(test_dic.items())
random.Random(4).shuffle(l_test)
l_test

In [ ]:

print(len(l_train),len(l_test),len(l_val))

In [ ]:
train_data = []
train_labels = []

for i in range(len(l_train)):
    file_name,label = l_train[i]
    img = cv2.imread(file_name)
    try:
        img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
        img = img.astype('float32')/255.0
        train_data.append(img)
        train_labels.append(label)

    except:
        print(i,file_name)
        print("Not possible")        

        
train_data = np.array(train_data)
print(train_data.shape)

train_labels = np.array(train_labels)
print(train_labels.shape)    
 
print('^_^-total data finished-^_^')  

In [ ]:
val_data = []
val_labels = []

for i in range(len(l_val)):
    file_name,label = l_val[i]
    img = cv2.imread(file_name)
    try:
        img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
        img = img.astype('float32')/255.0
        val_data.append(img)
        val_labels.append(label)

    except:
        print(i,file_name)
        print("Not possible")        

        
val_data = np.array(val_data)
print(val_data.shape)

val_labels = np.array(val_labels)
print(val_labels.shape)    
 
print('^_^-total data finished-^_^')  

In [ ]:

test_data = []
test_labels = []

for i in range(len(l_test)):
    file_name,label = l_test[i]
    img = cv2.imread(file_name)
    try:
        img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
        img = img.astype('float32')/255.0
        test_data.append(img)
        test_labels.append(label)

    except:
        print(i,file_name)
        print("Not possible")   
        
test_data = np.array(test_data)
print(test_data.shape)        
test_labels = np.array(test_labels)
print(test_labels.shape)   

In [ ]:

in_model = tf.keras.applications.VGG19(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet',classes = 2)

inputs = tf.keras.Input(shape=(224,224,3))
#inputs2 = tf.keras.Input(shape=(224,224)
#x=tf.keras.layers.LSTM(224,input_shape=X_train.shape[1:], return_sequences=True)(inputs)
x1 = in_model(inputs)
#x1 = Conv2D(512,(1,1),activation = 'relu')(x1)
print(x1.shape)


flat = Flatten()(x1)
dense_1 = Dense(4096,activation = 'relu')(flat)
#dense_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
#dense_2 = Dropout(0.5)(dense_2)
prediction = Dense(2,activation = 'softmax')(dense_2)

in_pred45 = Model(inputs = inputs,outputs = prediction)

In [ ]:
in_model2 = tf.keras.applications.DenseNet121(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet',classes = 2)

inputs = tf.keras.Input(shape=(224,224,3))
#inputs2 = tf.keras.Input(shape=(224,224)
#x=tf.keras.layers.LSTM(224,input_shape=X_train.shape[1:], return_sequences=True)(inputs)
x1 = in_model2(inputs)
#x1 = Conv2D(512,(1,1),activation = 'relu')(x1)
print(x1.shape)


flat = Flatten()(x1)
dense_1 = Dense(4096,activation = 'relu')(flat)
#dense_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
#dense_2 = Dropout(0.5)(dense_2)
prediction = Dense(2,activation = 'softmax')(dense_2)

in_pred = Model(inputs = inputs,outputs = prediction)

In [ ]:
in_model2 = tf.keras.applications.ResNet50(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet',classes = 2)

inputs = tf.keras.Input(shape=(224,224,3))
#inputs2 = tf.keras.Input(shape=(224,224)
#x=tf.keras.layers.LSTM(224,input_shape=X_train.shape[1:], return_sequences=True)(inputs)
x1 = in_model2(inputs)
#x1 = Conv2D(512,(1,1),activation = 'relu')(x1)
print(x1.shape)


flat = Flatten()(x1)
dense_1 = Dense(4096,activation = 'relu')(flat)
#dense_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
#dense_2 = Dropout(0.5)(dense_2)
prediction = Dense(2,activation = 'softmax')(dense_2)

in_pred4 = Model(inputs = inputs,outputs = prediction)

In [ ]:
in_pred4.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])
history1 = in_pred4.fit(train_data,train_labels,batch_size = 8,epochs = 1,validation_data = (val_data,val_labels))

In [ ]:
in_pred.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])
history2 = in_pred.fit(train_data,train_labels,batch_size = 8,epochs = 1,validation_data = (val_data,val_labels))

In [ ]:
in_pred45.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])
history3 = in_pred45.fit(train_data,train_labels,batch_size = 8,epochs = 1,validation_data = (val_data,val_labels))

In [ ]:
np.save('my_history1',history1.history)

In [ ]:
in_pred45.save_weights("VGG19_weights.h5")

In [ ]:
in_pred4.save_weights("resnet50_weights.h5")

In [ ]:
in_pred 0.33272948

In [ ]:
in_pred45.evaluate(test_data,test_labels)

In [ ]:
in_pred45.evaluate(val_data,val_labels)

In [ ]:
in_pred4.evaluate(test_data,test_labels)

In [ ]:
in_pred4.evaluate(val_data,val_labels)

In [ ]:
in_pred.evaluate(test_data,test_labels)

In [ ]:
in_pred.evaluate(val_data,val_labels)

In [ ]:
in_pred.save_weights("Densenet121_weights.h5")

In [ ]:
inpred = 0.33172

In [ ]:
inpred4 =  0.33354

In [ ]:
inpred45  = 0.33473

In [ ]:
    fuzzymeasure = np.array([0.33373, 0.33313, 0.33313])
    Ypred_fuzzy = np.zeros(shape = mobilenet2_test.shape, dtype = float)
    for sample in range(0,mobilenet2_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet2_test[sample][classes],resnet2_test[sample][classes],densenet2_test[sample][classes]])
            fuzzy = sugeno_fuzzy_integral_generalized(scores,fuzzymeasure,axis = 0) 
            Ypred_fuzzy[sample][classes] = fuzzy
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)

    acc = accuracy_score(test_labels,pred_label)
    print(acc)

In [ ]:
in_pred.load_weights("/gdrive/MyDrive/densenet121_new_weights (1).h5")
in_pred4.load_weights("/gdrive/MyDrive/Resnet50_new_weights.h5")
in_pred45.load_weights("/gdrive/MyDrive/xception_new_weights.h5")

In [ ]:
in_pred.load_weights("Densenet121_weights.h5")
in_pred4.load_weights("resnet50_weights.h5")
in_pred45.load_weights("VGG19_weights.h5")

In [ ]:
mobilenet_test = in_pred.predict(val_data)
resnet_test = in_pred4.predict(val_data)
densenet_test = in_pred45.predict(val_data)

In [ ]:
mobilenet2_test = in_pred.predict(test_data)
resnet2_test = in_pred4.predict(test_data)
densenet2_test = in_pred45.predict(test_data)

In [ ]:
mobilenet2_test.shape

In [ ]:
in_pred.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])
in_pred4.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])
in_pred45.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])

In [ ]:
!pip install mealpy==v1.0.4

In [ ]:
from mealpy.swarm_based.GWO import BaseGWO
from sklearn.metrics import accuracy_score
from sympy.solvers import solve
from sympy import Symbol

def objective(solution = None):
    solution = solution.reshape((-1))
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    #fuzzymeasures = np.array([0.09,0.08,0.07])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = scoreslambda[0] * fmlambda[0]

            for i in range(1,2):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = fuzzyprediction + scoreslambda[i] *(ge_curr - ge_prev)
                ge_prev = ge_curr

            fuzzyprediction = fuzzyprediction + scoreslambda[2] * ( 1 - ge_prev)
            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)

    acc = accuracy_score(val_labels,pred_label)
    return -acc

obj_func = objective

lb = [0]
ub = [1]
problem_size = 3
batch_size = 25
verbose = True
epoch = 30
pop_size = 100

pc = 0.95
pm = 0.025

md1 = BaseGWO(obj_func, lb, ub, problem_size, batch_size, verbose, epoch, pop_size)
best_pos1, best_fit1, list_loss1 = md1.train()
print(md1.solution[0])
print(md1.solution[1])

In [ ]:
    fuzzymeasures = np.array([1.11680804,0.40229502,1.0872415 ])
    #fuzzymeasures = np.array([0.09,0.08,0.07])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet2_test.shape, dtype = float)
    for sample in range(0,mobilenet2_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet2_test[sample][classes],resnet2_test[sample][classes],densenet2_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = scoreslambda[0] * fmlambda[0]

            for i in range(1,2):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = fuzzyprediction + scoreslambda[i] *(ge_curr - ge_prev)
                ge_prev = ge_curr

            fuzzyprediction = fuzzyprediction + scoreslambda[2] * ( 1 - ge_prev)
            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)

    acc = accuracy_score(test_labels,pred_label)

#Sugeno Integral

In [ ]:
from mealpy.swarm_based.WOA import BaseWOA
from sklearn.metrics import accuracy_score



def objective(solution=None):
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)
    acc = accuracy_score(val_labels,pred_label)
    #print(acc)
    return -acc
obj_func = objective


lb = [0]
ub = [1]
problem_size = 3
batch_size = 32
verbose = True
epoch = 30
pop_size = 50

pc = 0.95
pm = 0.025

md2 = BaseWOA(obj_func, lb, ub, problem_size, batch_size, verbose, epoch, pop_size)
best_pos1, best_fit1, list_loss1 = md2.train()
print(md2.solution[0])
print(md2.solution[1])

In [ ]:
from mealpy.swarm_based.GWO import BaseGWO
from sklearn.metrics import accuracy_score
from sympy.solvers import solve
from sympy import Symbol

def objective(solution = None):
    solution = solution.reshape((-1))
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    #fuzzymeasures = np.array([0.09,0.08,0.07])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)

    acc = accuracy_score(val_labels,pred_label)
    return -acc

obj_func = objective

lb = [0]
ub = [1]
problem_size = 3
batch_size = 25
verbose = True
epoch = 30
pop_size = 100

pc = 0.95
pm = 0.025

md1 = BaseGWO(obj_func, lb, ub, problem_size, batch_size, verbose, epoch, pop_size)
best_pos1, best_fit1, list_loss1 = md1.train()
print(md1.solution[0])
print(md1.solution[1])

In [ ]:
    fuzzymeasures = np.array([0.40918803,0.27254635,0.97002079])
    Ypred_fuzzy = np.zeros(shape = mobilenet2_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)

    acc = accuracy_score(test_labels,pred_label)
    print(acc)

In [ ]:
from mealpy.swarm_based.GWO import BaseGWO
from sklearn.metrics import accuracy_score



def objective(solution=None):
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)
    acc = accuracy_score(val_labels,pred_label)
    #print(acc)
    return -acc
obj_func = objective


lb = [0]
ub = [1]
problem_size = 3
batch_size = 16
verbose = True
epoch = 30
pop_size = 50

pc = 0.95
pm = 0.025

md2 = BaseGWO(obj_func, lb, ub, problem_size, batch_size, verbose, epoch, pop_size)
best_pos1, best_fit1, list_loss1 = md2.train()
print(md2.solution[0])
print(md2.solution[1])

In [ ]:
from mealpy.swarm_based.ALO import BaseALO
from sklearn.metrics import accuracy_score



def objective(solution=None):
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)
    acc = accuracy_score(val_labels,pred_label)
    #print(acc)
    return -acc
obj_func = objective


lb = [0]
ub = [1]
problem_size = 3
batch_size = 32
verbose = True
epoch = 100
pop_size = 200

pc = 0.95
pm = 0.025

md2 = BaseALO(obj_func, lb, ub, problem_size, batch_size, verbose, epoch, pop_size)
best_pos1, best_fit1, list_loss1 = md2.train()
print(md2.solution[0])
print(md2.solution[1])

In [ ]:
    fuzzymeasures = np.array([0.69415197,0.90141054,0.80896715])
    Ypred_fuzzy = np.zeros(shape = mobilenet2_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)

    acc = accuracy_score(test_labels,pred_label)
    print(acc)

In [ ]:
from mealpy.physics_based.SA import BaseSA
from sklearn.metrics import accuracy_score



def objective(solution=None):
    
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)
    acc = accuracy_score(val_labels,pred_label)
    #print(acc)
    return -acc
obj_func = objective

lb = [0]
ub = [1]
problem_size = 3
batch_size = 4
verbose = True
epoch = 100
pop_size = 100


pc = 0.9
pm = 0.02

md2 = BaseSA(obj_func, lb, ub, problem_size, batch_size, verbose, epoch, pop_size)
best_pos1, best_fit1, list_loss1 = md2.train()
print(md2.solution[0])
print(md2.solution[1])

In [ ]:
    fuzzymeasures = np.array([0.82761983 1.06492615 1.26400566])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)

    acc = accuracy_score(val_labels,pred_label)

In [ ]:
from mealpy.swarm_based.WOA import BaseWOA
from sklearn.metrics import accuracy_score



def objective(solution=None):
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)
    acc = accuracy_score(val_labels,pred_label)
    #print(acc)
    return -acc
obj_func = objective


lb = [0]
ub = [1]
problem_size = 3
batch_size = 16
verbose = True
epoch = 25
pop_size = 100

pc = 0.95
pm = 0.025

md2 = BaseWOA(obj_func, lb, ub, problem_size, batch_size, verbose, epoch, pop_size)
best_pos1, best_fit1, list_loss1 = md2.train()
print(md2.solution[0])
print(md2.solution[1])

In [ ]:
    fuzzymeasures = np.array([0.16065645,0.99115229,0.56586762])
    Ypred_fuzzy = np.zeros(shape = mobilenet2_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)

    acc = accuracy_score(test_labels,pred_label)

In [ ]:
from mealpy.evolutionary_based.DE import BaseDE
from sklearn.metrics import accuracy_score



def objective(solution=None):
    
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)
    acc = accuracy_score(val_labels,pred_label)
    #print(acc)
    return -acc
obj_func = objective

lb = [0]
ub = [1]
problem_size = 3
batch_size = 32
verbose = True
epoch = 100
pop_size = 150

pc = 0.95
pm = 0.025

md2 = BaseDE(obj_func, lb, ub, problem_size, batch_size, verbose, epoch, pop_size)
best_pos1, best_fit1, list_loss1 = md2.train()
print(md2.solution[0])
print(md2.solution[1])

In [ ]:
    fuzzymeasures = np.array([0.75523624,0.52702442,0.80056234])

    Ypred_fuzzy = np.zeros(shape = mobilenet2_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)

    acc = accuracy_score(test_labels,pred_label)
    print(acc)

In [ ]:
from mealpy.swarm_based.WOA import BaseWOA
from sklearn.metrics import accuracy_score



def objective(solution=None):
    
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
    for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)
    acc = accuracy_score(val_labels,pred_label)
    #print(acc)
    return -acc
obj_func = objective

lb = [0]
ub = [1]
problem_size = 3
batch_size = 32
verbose = True
epoch = 50
pop_size =50

pc = 0.95
pm = 0.025

md2 = BaseWOA(obj_func, lb, ub, problem_size, batch_size, verbose, epoch, pop_size)
best_pos1, best_fit1, list_loss1 = md2.train()
print(md2.solution[0])
print(md2.solution[1])

In [ ]:
from scipy.optimize import differential_evolution
KRR_alpha_lim = (0,1)
boundaries = [KRR_alpha_lim] + [KRR_alpha_lim] + [KRR_alpha_lim]
extra_variables = (val_data,val_labels)
number_of_classes = 2

def objective(hyperparameters,x,y):
    alpha,beta,gamma = hyperparameters
    fuzzymeasures = np.array([alpha,beta,gamma])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[1])
    Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
     for sample in range(0,mobilenet_test.shape[0]):
        for classes in range(0,2):
            scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
            permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
            scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
            fmlambda = fuzzymeasures[permutedidx]

            ge_prev = fmlambda[0]
            fuzzyprediction = np.min(scoreslambda[0], fmlambda[0])

            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = np.max(fuzzyprediction,np.min(scoreslambda[i],ge_curr))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)
    acc = accuracy_score(val_labels,pred_label)
    return -acc
    
solver = differential_evolution(objective,boundaries,args=(extra_variables)) 

In [ ]:
best_hyperparams = solver.x
best_rmse = solver.fun

In [ ]:
print(best_hyperparams,best_rmse)

In [ ]:
best_rmse

In [ ]:
    import numpy as np

    from sympy.solvers import solve
    from sympy import Symbol
    fuzzymeasures = np.array([0.69415197,0.90141054,0.80896715])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      lam = np.asarray(lam[2])

In [ ]:

lam

In [ ]:
mobilenet_test = np.asarray([[0.2,0.3,0.2]],dtype = float);
resnet_test = np.asarray([[0.1,0.2,0.7]],dtype = float);
densenet_test = np.asarray([[0.2,0.3,0.5]],dtype= float);



In [ ]:
Ypred_fuzzy = np.zeros(shape = mobilenet_test.shape, dtype = float)
for sample in range(0,mobilenet_test.shape[0]):
  for classes in range(0,2):
    scores = np.array([mobilenet_test[sample][classes],resnet_test[sample][classes],densenet_test[sample][classes]])
            #print(scores.shape)
    permutedidx = np.flip(np.argsort(scores))
            #print(permutedidx.shape)
    scoreslambda = scores[permutedidx]
            #print(scoreslambda.shape)
    fmlambda = fuzzymeasures[permutedidx]

    ge_prev = fmlambda[0]
    fuzzyprediction = np.minimum(scoreslambda[0], fmlambda[0])

    for i in range(1,3):
      ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
      fuzzyprediction = np.maximum(fuzzyprediction,np.minimum(scoreslambda[i],ge_curr))
      ge_prev = ge_curr

    Ypred_fuzzy[sample][classes] = fuzzyprediction


In [ ]:
Ypred_fuzzy

#generating labels

In [ ]:
densenet2_test = in_pred.predict(test_data)
resnet2_test = in_pred4.predict(test_data)
vgg2_test = in_pred45.predict(test_data)

#Generating predicted labels

In [ ]:
 
Y_test_densenet = np.argmax(densenet2_test, axis=1)
Y_test_vgg = np.argmax(vgg2_test, axis=1)
Y_test_resnet = np.argmax(resnet2_test, axis=1)
Y_pred= np.argmax(test_labels, axis=1)

#average ensembeling

In [ ]:
y_pred = np.average([Y_test_densenet,Y_test_vgg,Y_test_resnet],axis = 0)

In [ ]:
y_pred = np.around(y_pred).astype(int)

In [ ]:
y_pred

In [ ]:
print(accuracy_score(Y_pred,y_pred))
print(confusion_matrix(Y_pred,y_pred))
print(classification_report(Y_pred,y_pred))

#weighted average using val accuracies


In [ ]:
y_pred_weight = (Y_test_vgg * inpred45 + Y_test_resnet * inpred4 + Y_test_densenet * inpred)
#y_pred_weight = y_pred_weight.astype(int)

In [ ]:
y_pred_weight

In [ ]:
y_pred_weight = np.around(y_pred_weight).astype(int)

In [ ]:
y_pred_weight

In [ ]:
Y_pred

In [ ]:
print(accuracy_score(Y_pred,y_pred_weight))
print(classification_report(Y_pred,y_pred_weight))
print(confusion_matrix(Y_pred,y_pred_weight))

#Majority Voting

In [ ]:
count = 0 
total = 0
test_pred = Y_test_resnet
for i in range(len(Y_pred)):
    if Y_pred[i] == Y_test_densenet[i]:
        k = Y_test_densenet[i]
        count = count+1
    if Y_pred[i] == Y_test_resnet[i]:
        k = Y_test_resnet[i]
        count = count+1
    if Y_pred[i] == Y_test_vgg[i]:   
        k = Y_test_vgg[i]  
        count = count+1
 
    if count == 0 or count == 1:
        if k == test_pred[i] and k == 1:
            test_pred[i] = test_pred[i]-1
        elif k == test_pred[i] and k == 0:
            test_pred[i] = test_pred[i]+1
    if count == 2 or count == 3:
        test_pred[i] = k
        total = total+1
    count = 0

In [ ]:
test_pred[1000:1400]

In [ ]:
test_pred = np.around(test_pred).astype(int)

In [ ]:
print(accuracy_score(Y_pred,test_pred))
print(classification_report(Y_pred,test_pred))
print(confusion_matrix(Y_pred,test_pred))
